In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import mmcv
import importlib
import torch
import torchvision.transforms as transforms
from mmcv import Config
from mmcls.datasets.pipelines import Compose
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from PIL import Image
import heapq
from mmseg.apis import inference_segmentor, init_segmentor
from mmseg.core.evaluation import get_palette
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2

import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

from scripts import generate_statistics_single
from scripts import visualization_seg_masks
from scripts import generate_statistics
from scripts import generate_cams
from scripts import utils
from scripts.utils.pipeline import get_pipeline_torchvision
from scripts import generate_segs

In [2]:
configSeg = '../segmentation/ocrnet_hr48_512x512_carparts_noflip/ocrnet_hr48_512x512_carparts_noflip.py'
checkpointSeg = '../segmentation/ocrnet_hr48_512x512_carparts_noflip/latest.pth'
imgRootTrain = '../data/CompCars_sv_original_split/train'
imgRootVal = '../data/CompCars_sv_original_split/val'
configCAMResOG = '../CAMModels/resnet/compCars_Original/resnet50_b128x2_compcars-original-split.py'
checkpointCAMResOG = '../CAMModels/resnet/compCars_Original/latest.pth'
configCAMResCol = '../CAMModels/resnet/compCars_Color/resnet50_b128x2_compcars-color-split.py'
checkpointCAMResCol = '../CAMModels/resnet/compCars_Color/latest.pth'
configCAMResWeb = '../CAMModels/resnet/compCars_Web/resnet50_b128x2_compcars-web-original-split.py'
checkpointCAMResWeb = '../CAMModels/resnet/compCars_Web/latest.pth'

configCAMSwinSmallOG = '../CAMModels/swinSmall/compCars_Original/swin-small_b128x2_compcars-original-split.py'
checkpointCAMSwinSmallOG = '../CAMModels/swinSmall/compCars_Original/latest.pth'
configCAMSwinSmallCol = '../CAMModels/swinSmall/compCars_Color/swin-small_b128x2_compcars-color-split.py'
checkpointCAMSwinSmallCol = '../CAMModels/swinSmall/compCars_Color/latest.pth'
configCAMSwinSmallWeb = '../CAMModels/swinSmall/compCars_WebOriginal/swin-small_b128x2_compcars-web-original-split.py'
checkpointCAMSwinSmallWeb = '../CAMModels/swinSmall/compCars_WebOriginal/latest.pth'

configCAMSwinBaseOG = '../CAMModels/swinBase/compCars_Original/swin-base_b128x2_compcars-original-split.py'
checkpointCAMSwinBaseOG = '../CAMModels/swinBase/compCars_Original/latest.pth'
configCAMSwinBaseCol = '../CAMModels/swinBase/compCars_Color/swin-base_b128x2_compcars-color-split.py'
checkpointCAMSwinBaseCol = '../CAMModels/swinBase/compCars_Color/latest.pth'
configCAMSwinBaseWeb = '../CAMModels/swinBase/compCars_WebOriginal/swin-base_b128x2_compcars-web-original-split.py'
checkpointCAMSwinBaseWeb = '../CAMModels/swinBase/compCars_WebOriginal/latest.pth'

annfileTrain = '../annfileTrain.txt'
annfileVal = '../annfileVal.txt'
annfileValGt = '../annfileValGt.txt'
imgNameTrain='Acura_Acura_ILX_2a6cce617fc27d.jpg'
imgNameVal = 'Acura_Acura_ILX_3c11ef9e42931b.jpg'
imgPathTrain=os.path.join(imgRootTrain, imgNameTrain)
imgPathVal=os.path.join(imgRootVal, imgNameVal)

paletteName='Comp_Original_Ocrnet_Carparts_Noflip'

In [3]:
imgRootValOriginal = '../data/CompCars_sv_original_split/val'
imgRootValColor = '../data/CompCars_sv_color_split/val'
imgRootValWeb = '../data/CompCars_web_original_split/val'

In [5]:
igR = imgRootValOriginal
cfgC = configCAMResOG
ckpC = checkpointCAMResOG
svPre = '../preGenData/CompCars_Original/ResNet/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMs', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsXGrad', '--method', 'xgradcam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGrad', '--method', 'eigengradcam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayer', '--method', 'layercam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGPU', '--device', 'cuda', '--vit-like'])

Method for CAM generation: GradCAM, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13379/13379, 3.3 task/s, elapsed: 4027s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData/CompCars_Color/SwinSmall\valAllCAMs.npz
Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13379/13379, 3.2 task/s, elapsed: 4177s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData/CompCars_Color/SwinSmall\valAllCAMsGrad++.npz
Method for CAM generation: xgradcam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13379/13379, 3.1 task/s, elapsed: 4307s, ETA:     0sS

In [5]:
igR = imgRootValOriginal
cfgC = configCAMResOG
ckpC = checkpointCAMResOG
svPre = '../preGenData/CompCars_Original/ResNet/GPU/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsXGradGPU', '--method', 'xgradcam', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGradGPU', '--method', 'eigengradcam', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--device', 'cuda'])

Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 20.6 task/s, elapsed: 647s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData/CompCars_Original/ResNet/GPU\valAllCAMsGrad++GPU.npz
Method for CAM generation: xgradcam, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 23.0 task/s, elapsed: 580s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData/CompCars_Original/ResNet/GPU\valAllCAMsXGradGPU.npz
Method for CAM generation: eigencam, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.5 task/s, elaps

In [ ]:
igR = imgRootValOriginal
cfgC = configCAMSwinSmallOG
ckpC = checkpointCAMSwinSmallOG
svPre = '../preGenData/CompCars_Original/ResNet/GPU/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsXGradGPU', '--method', 'xgradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGradGPU', '--method', 'eigengradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like', '--device', 'cuda'])

In [ ]:
igR = imgRootValOriginal
cfgC = configCAMSwinBaseOG
ckpC = checkpointCAMSwinBaseOG
svPre = '../preGenData/CompCars_Original/ResNet/GPU/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsXGradGPU', '--method', 'xgradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGradGPU', '--method', 'eigengradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like', '--device', 'cuda'])